In [1]:
# TAO PeopleNet 관련 패키지 설치
!pip install ultralytics torch torchvision
!pip install opencv-python pillow matplotlib
!pip install onnx onnxruntime-gpu



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision.models import detection
import cv2
import numpy as np
import matplotlib.pyplot as plt

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 디바이스: {device}")

# 사전 훈련된 사람 감지 모델 (Faster R-CNN)
model = detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()

print("TAO PeopleNet 스타일 모델 로드 완료!")
print("RTX 3090에서 실행 중... 🚀")


사용 중인 디바이스: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


TAO PeopleNet 스타일 모델 로드 완료!
RTX 3090에서 실행 중... 🚀


In [3]:
def detect_people(image_path_or_array):
    """TAO PeopleNet 스타일 사람 감지 함수"""
    
    # 이미지 로드 및 전처리
    if isinstance(image_path_or_array, str):
        image = cv2.imread(image_path_or_array)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path_or_array
    
    # PyTorch 텐서로 변환
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])
    
    input_tensor = transform(image).unsqueeze(0).to(device)
    
    # 추론 실행
    with torch.no_grad():
        predictions = model(input_tensor)
    
    # 사람(클래스 1) 감지 결과 필터링
    scores = predictions[0]['scores'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    boxes = predictions[0]['boxes'].cpu().numpy()
    
    # 사람만 필터링 (COCO dataset에서 사람은 클래스 1)
    person_mask = labels == 1
    person_scores = scores[person_mask]
    person_boxes = boxes[person_mask]
    
    # 신뢰도 0.5 이상만 선택
    high_conf_mask = person_scores > 0.5
    final_boxes = person_boxes[high_conf_mask]
    final_scores = person_scores[high_conf_mask]
    
    return final_boxes, final_scores

# 테스트용 샘플 이미지 생성 (사람이 있는 이미지)
def create_sample_image():
    """테스트용 이미지 생성"""
    img = np.zeros((400, 600, 3), dtype=np.uint8)
    
    # 배경
    img[:, :] = [100, 150, 200]
    
    # 사람 형태 그리기 (간단한 직사각형들)
    # 머리
    cv2.circle(img, (200, 80), 25, (255, 220, 177), -1)
    # 몸통
    cv2.rectangle(img, (175, 105), (225, 200), (0, 100, 200), -1)
    # 팔
    cv2.rectangle(img, (140, 120), (175, 140), (255, 220, 177), -1)
    cv2.rectangle(img, (225, 120), (260, 140), (255, 220, 177), -1)
    # 다리
    cv2.rectangle(img, (185, 200), (200, 280), (0, 0, 150), -1)
    cv2.rectangle(img, (210, 200), (225, 280), (0, 0, 150), -1)
    
    return img

print("TAO PeopleNet 함수 준비 완료!")



TAO PeopleNet 함수 준비 완료!
